In [ ]:
import gradio as gr
import numpy as np
from scipy.special import softmax
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = "@user" if t.startswith("@") and len(t) > 1 else t
        t = "http" if t.startswith("http") else t
        new_text.append(t)
    return " ".join(new_text)


def predict(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors="pt")
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    result_data = []
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        max_index = np.argmax(scores[ranking[i]])
        max_label = config.id2label[ranking[max_index]]
        result_data.append(f"{i+1}) {l} {np.round(float(s), 3)}")

        if max_label == "negative":
            response = "😟 - It's a toxic message. It can impact your mental health."
        elif max_label == "positive":
            response = "😊 - It's positive message. All should be fine with reading this message."
        else:
            response = "😐 - This seems to be a neutral message. It's hard for me to rate this message if it's either good or bad."

    return "\n".join(result_data), response

In [ ]:
demo = gr.Interface(fn=predict, inputs="text", outputs=["text", "text"])

if __name__ == "__main__":
    demo.launch(share=True, inbrowser=True)